In [5]:
# import os
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# import pandas as pd
# import time

# # Selenium WebDriver 설정
# options = webdriver.ChromeOptions()
# options.add_argument('--disable-gpu')  # GPU 사용 비활성화
# options.add_argument('--no-sandbox')
# driver = webdriver.Chrome(options=options)

# # 크롤링할 URL
# url = "https://www.vivino.com/users/sippingfinewine"  # 실제 URL로 변경
# driver.get(url)

# # 페이지 로드 대기
# time.sleep(3)

# # 사용자 이름 가져오기 (xpath를 사용)
# try:
#     user_name = driver.find_element(By.XPATH, "//div[@class='user-header__name header-medium bold']").text.strip()
# except Exception as e:
#     print("User name not found")
#     user_name = "Unknown User"

# # 와인 평가 데이터를 저장할 리스트
# wine_ratings = []

# # "Show More" 버튼을 클릭하면서 데이터 크롤링
# def load_and_scrape(driver, user_name, pause_time=2):
#     while True:
#         # 와인 평가 데이터 크롤링
#         try:
#             wine_cards = driver.find_elements(By.XPATH, "//div[@class='user-activity-item']")  # 각 와인 평가 데이터를 담은 div
#             for wine_card in wine_cards:
#                 try:
#                     # 와인 이름
#                     wine_name = wine_card.find_element(By.XPATH, ".//p[@class='wine-name header-medium']/a").text.strip()
                    
#                     # 사용자 평가 점수 (별의 갯수)
#                     user_rating_stars = wine_card.find_elements(By.XPATH, ".//span[@class='rating rating-xs text-inline-block']/i[contains(@class, 'icon-100-pct')]")
#                     user_rating = len(user_rating_stars)  # 'icon-100-pct' 클래스의 개수

#                     # 평균 평점
#                     try:
#                         avg_rating = wine_card.find_element(By.XPATH, ".//span[@class='header-large text-block']").text.strip()
#                     except Exception:
#                         avg_rating = "N/A"

#                     # 평가 데이터 저장
#                     wine_ratings.append({
#                         "user_name": user_name,
#                         "wine_name": wine_name,
#                         "user_rating": user_rating,
#                         "avg_rating": avg_rating
#                     })
#                 except Exception as e:
#                     continue  # 데이터가 없으면 무시
#         except Exception as e:
#             print("No wine cards found.")
#             break  # 더 이상 데이터를 찾을 수 없으면 종료

#         # "Show More" 버튼 클릭
#         try:
#             show_more_button = driver.find_element(By.XPATH, "//button[@id='btn-more']")  # "Show More" 버튼
#             driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)  # 버튼으로 스크롤
#             time.sleep(3)  # 스크롤 후 대기
#             show_more_button.click()  # 버튼 클릭
#             time.sleep(pause_time)  # 데이터 로딩 대기
#         except Exception as e:
#             print("No more 'Show More' button found or unable to click. Stopping.")
#             break  # 버튼이 없으면 크롤링 종료

# # 데이터를 로드하며 크롤링
# load_and_scrape(driver, user_name)

# # 데이터를 CSV로 저장
# if wine_ratings:
#     output_file = os.path.expanduser("~/vivino_wine_ratings.csv")  # 저장 파일 경로
#     df = pd.DataFrame(wine_ratings)
#     df.to_csv(output_file, index=False)
#     print(f"Data saved to {output_file}")
# else:
#     print("No data to save.")

# # Selenium WebDriver 종료
# driver.quit()


No more 'Show More' button found or unable to click. Stopping.
Data saved to C:\Users\0730b/vivino_wine_ratings.csv


In [4]:
import os
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--force-device-scale-factor=0.5')
driver = webdriver.Chrome(options=options)

# follower_urls.csv 파일에서 URL 읽기
input_csv_file = os.path.expanduser("./follower_urls_d.csv")  # follower_urls.csv 경로
url_data = pd.read_csv(input_csv_file)  # profile_url 컬럼이 있다고 가정
url_data = url_data.head(1000)  # 상위 1000명의 사용자만 선택
all_reviews = []  # 모든 리뷰 데이터를 저장할 리스트
checkpoint_file = os.path.expanduser("C:./reviews_checkpoint_top1000.csv")  # 중간 저장 파일 경로

# 와인 평가 데이터를 크롤링하는 함수
def load_and_scrape(driver, user_url, max_reviews=1000, pause_time=2):
    wine_ratings = []
    driver.get(user_url)
    time.sleep(3)

    # 사용자 이름 가져오기
    try:
        user_name = driver.find_element(By.XPATH, "//div[@class='user-header__name header-medium bold']").text.strip()
    except NoSuchElementException:
        print(f"User name not found for URL: {user_url}")
        user_name = "Unknown User"

    # "Show More" 버튼 클릭하며 데이터 수집
    collected_ids = set()  # 이미 수집된 리뷰 ID 추적
    while True:
        try:
            wine_cards = driver.find_elements(By.XPATH, "//div[@class='user-activity-item']")
            if not wine_cards and len(wine_ratings) == 0:
                print(f"No reviews found for user: {user_name}")
                break  # 리뷰가 없으면 중단

            for wine_card in wine_cards:
                review_id = wine_card.get_attribute("id")  # 고유 ID 가져오기
                if review_id in collected_ids:
                    continue  # 이미 처리한 리뷰는 건너뜀

                collected_ids.add(review_id)  # 고유 ID 저장

                try:
                    wine_name = wine_card.find_element(By.XPATH, ".//p[contains(@class, 'wine-name')]/a").text.strip()
                except NoSuchElementException:
                    wine_name = "Unknown Wine"

                try:
                    winery_name = wine_card.find_element(By.XPATH, ".//div[@class='wine-info']//span[@class='text-small']/a").text.strip()
                except NoSuchElementException:
                    winery_name = "Unknown Winery"

                try:
                    vintage_year = wine_card.find_element(By.XPATH, ".//p[contains(@class, 'wine-name')]").text.split()[-1].strip()
                except NoSuchElementException:
                    vintage_year = ""

                # 사용자 평가 점수
                try:
                    stars = wine_card.find_elements(By.XPATH, ".//span[@class='rating rating-xs text-inline-block']/i")
                    user_rating = 0
                    for star in stars:
                        if "icon-100-pct" in star.get_attribute("class"):
                            user_rating += 1
                        elif "icon-50-pct" in star.get_attribute("class"):
                            user_rating += 0.5
                except NoSuchElementException:
                    user_rating = "N/A"

                try:
                    avg_rating = wine_card.find_element(By.XPATH, ".//span[@class='header-large text-block']").text.strip()
                except NoSuchElementException:
                    avg_rating = "N/A"

                wine_ratings.append({
                    "user_name": user_name,
                    "wine_name": f"{winery_name} {wine_name} {vintage_year}".strip(),
                    "user_rating": user_rating,
                    "avg_rating": avg_rating,
                    "profile_url": user_url
                })

            try:
                show_more_button = driver.find_element(By.XPATH, "//button[@id='btn-more-activities']")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
                time.sleep(1)
                show_more_button.click()
                time.sleep(pause_time)
            except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
                print(f"No more 'Show More' button for {user_name}. Stopping.")
                break

        except Exception as e:
            print(f"Error during scraping for {user_name}: {e}")
            break

        if len(wine_ratings) >= max_reviews:
            break

    return wine_ratings

# 중간 저장 함수
def save_checkpoint(data, file_path):
    df = pd.DataFrame(data)
    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)
        df = pd.concat([existing_df, df], ignore_index=True)
    df.to_csv(file_path, index=False)
    print(f"Checkpoint saved to {file_path}")

# 상위 1000명의 URL에 대해 리뷰 수집
for index, row in url_data.iterrows():
    user_url = row['profile_url']  # follower_urls.csv의 'profile_url' 컬럼
    print(f"Scraping reviews for URL: {user_url}")
    try:
        user_reviews = load_and_scrape(driver, user_url, max_reviews=1000)
        all_reviews.extend(user_reviews)

        # 중간 저장
        if len(all_reviews) >= 200:
            save_checkpoint(all_reviews, checkpoint_file)
            all_reviews = []  # 중간 저장 후 메모리 데이터 초기화

    except Exception as e:
        print(f"Error occurred for {user_url}: {e}")
        continue

# 남은 데이터를 저장
if all_reviews:
    save_checkpoint(all_reviews, checkpoint_file)

# Selenium WebDriver 종료
driver.quit()

Scraping reviews for URL: https://www.vivino.com/users/d-avid.wolff
No reviews found for user: D avid Wolff
Scraping reviews for URL: https://www.vivino.com/users/m7b99bb78b8a888e94ef3e7771016d34
